# Soft Computing

## Cvičení 2 - Rozhodovací stromy (Decision Trees)

**K čemu se využívá?**

Metoda učení s učitelem, která se využívá pro klasifikaci (klasifikační stromy) a regresi (regresní stromy). Látka má přesah i do oblasti dolování dat (data mining), kde se využívá k predikci hodnoty proměnné. Během učení modelu se rozhodovací strom učí rozhodovací pravidla pro dělení dat do tříd.

**Princip klasifikace**

<img src="SOC2_soubory/decisiontree.jpg" style="height:350px">

<table>
    <thead>
        <th>Barva</th><th>Váha</th><th>Třída</th>
    </thead>
    <tr>
        <td>Hnědá</td><td>500</td><td>Mědved</td>
    </tr>
    <tr>
        <td>Šedivá</td><td>60</td><td>Vlk</td>
    </tr>
    <tr>
        <td>Šedivá</td><td>4000</td><td>Slon</td>
    </tr>
    <tr>
        <td>Žlutá</td><td>200</td><td>Tygr</td>
    </tr>
    <tr>
        <td>Hnědá</td><td>600</td><td>Mědved</td>
    </tr>
</table>

Cílem je nalézt taková pravidla (např.: Váha > 300), které maximalizují po rozdělení na subpopulace informační zisk (nebo jiný vybraný atribut). Pro prvotní rozdělení bude použito pravidlo s nejvyšším ziskem. Po získání informačního zisku klesne entropie. Cílem dalšího dělení subpopulací na menší subpopulace je minimalizace entropie.

**Proces klasifikace**
1. Získání datové sady
2. Dělení datové sady na trénovací a testovací
3. Vyber nejlepší atribut pro dělení na menší datové sady (informační zisk, Giniho index, ...)
4. Rozděl datovou sadu na menší datové sady rozhodovacím pravidlem
5. Rekurzivně opakuj bod 3 pro všechny potomky děleného uzlu
6. Otestuj model na testovací sadě pomocí metrik
7. Implementace modelu do kódu

**Metody výběru atributu**
1. Informační zisk = Rozdíl entropie před dělením datové sady a po rozdělení datové sady
2. Poměr zisku = Preference atributů s velkým počtem rozdílných hodnot
3. Giniho index = Poměr rozdělených prvků do větví (0 až 0.5), tzv. míra čistoty

**Hyperparametry**
1. Kritérium pro dělení na subpopulace (gini, entropy (=informační zisk))
2. Rozdělovací strategie: best, random
3. Maximální hloubka stromu (max_depth)
4. Minimální počet vzorků, potřebných pro další dělení (min_samples_split=2)
5. Minimální počet vzorků, které musí být v uzlu (min_samples_leaf=1)
6. Kolek atributů je z celkového počtu atributů zkoumáno pro rozhodování (max_features)
7. Odstranění větví s malou rozhodovací silou, tzv. Pruning (pre-pruning, post-pruning)
8. 

**Náhodné lesy**

Různé stromy mohou říct o různých prvcích, že náleží do různorodých tříd. Pro vyřešení této nejasnosti klasifikace se vytváří tzv. Náhodné lesy, které představují soustavu rozhodovacích stromů, kde každý strom provede zvlášť klasifikaci. Model lesa následně vybere na základě nejčastější klasifikaci jednu ze tříd. Tento přístup ke strojového učení používá například Kinect k Xbox360, kde les rozhoduje, která část těla představuje nohy, ruce, trup, hlavu, atd.

### Teorie

**Zadání**
1. Stáhněte si datovou sadu pro nějaká medicínská data (např.: charakteristiky nádorů)
2. Vytvořte rozhodovací strom pro klasifikaci a nechte mu základní parametry
3. Natrénujte model pomocí trénovacíh dat
4. Ověřte přesnost klasifikace pomocí vhodné metriky na testovacíh datech
5. Porovnejte přesnost klasifikace pomocí volby různorodých hyperparametrů
6. Porovnejte přesnost klasifikace pomocí náhodného lesu

In [ ]:
from sklearn.datasets import load_breast_cancer()

: 

#### 2.1 Klasifikační strom (from scratch)

**vysvětlení atributů z datové sady**

In [ ]:
#x1 = Wavelet transformed image variace
#x2 = Wavelet transformed image skewness
#x3 = Wavelet transformed image kurtosis
#x4 = image entropy
#x5 = třída
bank_note = [3.6216, 8.6661, -2.8073, -0.44699, 0]

#### 2.2 Klasifikační strom pomocí sklearn

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

In [ ]:
X, y = make_blobs(n_samples=300, center=4, random_state=0, cluster_std=1.0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow')

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier().fit(X, y)

In [ ]:
def visualize_classifier(model, X, y, ax=None, cmap='rainbow'):
    ax = ax or plt.gca()
    ax.scatter(X[:,0], X[:,1], c=y, s=30, cmap=cmap, clim=(y.min(), y.max()), zorder=3)
    ax.axis('tight')
    ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    model.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(*xlim, num=200), np.linspace(*ylim, num=200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    n_classes = len(np.unique(y))
    contours = ax.contourf(xx, yy, Z, alpha=0.3, levels=np.arrange(n_classes+1)-0.5, 
                            cmap=cmap, clim=(y.lim(), y.max()), zorder=1)
    ax.set(xlim=xlim, ylim=ylim)

visualize_classifier(DecisionTreeClassifier(), X, y)

#### 2.3 Klasifikační náhodný les

In [ ]:
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

In [ ]:
tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8, random_state=1)
bag.fit(X, y)
visualize_classifier(bag, X, y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=0)
visualize_classifier(model, X, y)

#### 2.4 Regresní náhodný les

#### 2.5 MNIST klasifikace

In [ ]:
from sklearn.datasets import load_digits

In [ ]:
digits = load_digits()
digits.keys()

In [ ]:
fig = plt.figure(figsize=(6, 6))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

### On-site cvičení

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
import graphviz 

nahrání datové sady

In [ ]:
data = load_breast_cancer()

explorační analýza

In [ ]:
X = data.data
y = data.target
y_names = data.target_names
a_names = data.feature_names
print(X)
print(y)
print(y_names)
print(a_names)

rozdělení datové sady na testovací a trénovací

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1./3., shuffle=True)
print(X_train)
print(y_train)

instantizace modelu

In [ ]:
model = DecisionTreeClassifier(
    #random_state=1,
    criterion="entropy",         #{gini, entropy}, default = gini
    splitter="best",             #{best, random}, default = best
    max_depth=None,              #default = None
    min_samples_split=2,         #default = 2
    min_samples_leaf=1,          #default = 1
    max_features="auto",           #{int, float, auto, sqrt, log2}, default=None
    max_leaf_nodes=None,         #default=None
    min_impurity_decrease=0.0    #default=0.0
)

trénování modelu

In [ ]:
model.fit(X_train, y_train)

validace modelu

In [ ]:
score = model.score(X_test, y_test)
print(score)

vizualizace modelu

In [ ]:
dot_data = tree.export_graphviz(
    model, 
    out_file=None,
    feature_names=a_names,
    class_names=y_names,
    filled=True, 
    rounded=True,
    special_characters=True
)
graph = graphviz.Source(dot_data)
graph.render("Breast Cancer")

In [ ]:
!pip install dtreeviz

In [ ]:
from dtreeviz.trees import dtreeviz

viz = dtreeviz(model, X_train, y_train, feature_names=a_names, class_names=list(y_names))
viz.save("Breast Cancer.svg")

náhodný les

In [ ]:
model = RandomForestClassifier(
    n_estimators=100,            #default = 100
    bootstrap=True,              #default = False
    n_jobs=1,                    #default = 1
    warm_start=False,            #default = False
    max_samples=None,            #default = None
    criterion="entropy",         #{gini, entropy}, default = gini
    max_depth=None,              #default = None
    min_samples_split=2,         #default = 2
    min_samples_leaf=1,          #default = 1
    max_features="auto",         #{int, float, auto, sqrt, log2}, default=None
    max_leaf_nodes=None,         #default=None
    min_impurity_decrease=0.0    #default=0.0
)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
score = model.score(X_test, y_test)
print(score)